El objetivo de este notebook es hacer una implementación rápida de MLP y ganar intuición sobre la forma de tomar las decisiones.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/repo_tesis/entorno_tesis_Molina"
!source bin/activate

Mounted at /content/drive
/content/drive/My Drive/repo_tesis/entorno_tesis_Molina


In [2]:
!pip install comet_ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 655.7/655.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 979.1/979.1 kB 59.3 MB/s eta 0:00:00
  Attempting uninstall: python-box
    Found existing installation: python-box 7.1.1
    Uninstalling python-box-7.1.1:
      Successfully uninstalled python-box-7.1.1


In [3]:
!pip install scikeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 33.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.


In [8]:
import numpy as np
import joblib
import sys
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from comet_ml import Experiment
import scikeras
import keras
from scikeras.wrappers import KerasClassifier
import tensorflow as tf

In [5]:
# Importar los datos
features = joblib.load("/content/drive/My Drive/repo_tesis/src/datos_definitivos/FEATURES_W200_I50.joblib")
label = joblib.load("/content/drive/My Drive/repo_tesis/src/datos_definitivos/label_W200_I50.joblib")

features_array = np.array(features)
label_array = np.array(label)

In [7]:
# elijo esta particion en particular para hacer la prueba

%cd "/content/drive/My Drive/repo_tesis/src/ronda_experimentos_1/particiones_1subTest"

folds_ind = joblib.load('folds_W200_I50_sujetoTest_' + str(1) + '.pkl')
sujetos_array = joblib.load('arraySujetos_folds_W200_I50_sujetoTest_' + str(1) + '.pkl')

print(folds_ind)
print(sujetos_array)

/content/drive/My Drive/repo_tesis/src/ronda_experimentos_1/particiones_1subTest
[array([   0,    1,    2, ..., 5555, 5556, 5557]), array([ 5558,  5559,  5560, ..., 12118, 12119, 12120]), array([12121, 12122, 12123, ..., 17671, 17672, 17673]), array([17674, 17675, 17676, ..., 24621, 24622, 24623]), array([24624, 24625, 24626, ..., 31115, 31116, 31117]), array([31118, 31119, 31120, ..., 37990, 37991, 37992]), array([37993, 37994, 37995, ..., 44782, 44783, 44784]), array([44785, 44786, 44787, ..., 51319, 51320, 51321]), array([51322, 51323, 51324, ..., 57842, 57843, 57844]), array([57845, 57846, 57847, ..., 64523, 64524, 64525])]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


# Entrenamiento y resultados en test

In [ ]:
# Elijo como particionar los datos

# armo una lista del largo de los datos, en el que indico a que fold corresponde cada uno
# por el criterio elegido, se que el primer fold es para test, asi que lo saco
fold_test = folds_ind[0]

# me elijo un fold para validacion para esta prueba
fold_val = folds_ind[1]

# ahora lo quito para no armar entrevero
folds_ind = folds_ind[2:]
sujetos_train = sujetos_array[2:]   # me quedo solo con los sujetos que voy usar para train

In [ ]:
tf.random.set_seed(42)

arch = [250,500,250]
catNum = 13
featureNum = 16
inputDim = featureNum * 15 # 15 es la cantidad de features

# layers = [tf.keras.layers.Dense(arch[0], activation='relu', input_shape=(featureNum,))]
# for i in range(1,len(arch)-1):
#     layers.append(tf.keras.layers.Dense(arch[i], activation='relu'))
# layers.append(tf.keras.layers.Dense(catNum, activation='softmax'))
# self.clf = tf.keras.Sequential(layers)
# self.clf.compile(optimizer='adam',
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
#     metrics=['accuracy'])

model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=[inputDim, ]))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))
